In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

22/09/20 15:36:40 WARN Utils: Your hostname, Jens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.38 instead (on interface en0)
22/09/20 15:36:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/20 15:36:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pyspark.sql.functions as F

In [3]:
#Creating pandas dataframe to later compare with a Spark dataframe:
import pandas as pd
import numpy as np

np.random.seed(456)

pandas_dataframe = pd.DataFrame(
        dict(n=np.arange(20), group=np.random.choice(list('abc'), 20))
)

pandas_dataframe

,n,group
0,0,b
1,1,b
2,2,c
3,3,a
4,4,c
5,5,c
6,6,a
7,7,b
8,8,a
9,9,b


In [4]:
#converting the pandas dataframe into Spark df:
df = spark.createDataFrame(pandas_dataframe)

df

DataFrame[n: bigint, group: string]

In [5]:
#Spark needs to be explicitly told to show the dataframe, visually:
df.show(5)

+---+-----+
|  n|group|
+---+-----+
|  0|    b|
|  1|    b|
|  2|    c|
|  3|    a|
|  4|    c|
+---+-----+
only showing top 5 rows



In [6]:
#trying out describe
df.describe()

DataFrame[summary: string, n: string, group: string]

In [7]:
df.describe().show()

+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+



In [8]:
#trying with more data:
from pydataset import data

mpg = spark.createDataFrame(data('mpg'))

mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [9]:
#normally, calling a column in padas would output a dataframe of just that column, however, if not specified Spark will 
#only clarify what object you just requested:
mpg.hwy

Column<'hwy'>

In [10]:
#One way to use columns as objects is by using .select method and calling in other columns that you only want to see:
mpg.select(mpg.hwy, mpg.cty, mpg.model)

DataFrame[hwy: bigint, cty: bigint, model: string]

In [11]:
#now let's show it as a df:
mpg.select(mpg.hwy, mpg.cty, mpg.model).show(10)


+---+---+----------+
|hwy|cty|     model|
+---+---+----------+
| 29| 18|        a4|
| 29| 21|        a4|
| 31| 20|        a4|
| 30| 21|        a4|
| 26| 16|        a4|
| 26| 18|        a4|
| 27| 18|        a4|
| 26| 18|a4 quattro|
| 25| 16|a4 quattro|
| 28| 20|a4 quattro|
+---+---+----------+
only showing top 10 rows



In [12]:
#we can also add to the columns specifically using aggregates and operations:
mpg.hwy+1

Column<'(hwy + 1)'>

In [13]:
mpg.select(mpg.hwy+1).show(5)

+---------+
|(hwy + 1)|
+---------+
|       30|
|       30|
|       32|
|       31|
|       27|
+---------+
only showing top 5 rows



In [14]:
#in Spark, to rename a column object, we use .alias:
mpg.select(mpg.hwy.alias('highway_mileage')).show(5)

+---------------+
|highway_mileage|
+---------------+
|             29|
|             29|
|             31|
|             30|
|             26|
+---------------+
only showing top 5 rows



In [15]:
#how to store column object and reference them:
col1= mpg.hwy.alias('highway_mileage')
col2= (mpg.hwy / 2).alias('highway_mileage_halved')
mpg.select(col1, col2).show(5)

+---------------+----------------------+
|highway_mileage|highway_mileage_halved|
+---------------+----------------------+
|             29|                  14.5|
|             29|                  14.5|
|             31|                  15.5|
|             30|                  15.0|
|             26|                  13.0|
+---------------+----------------------+
only showing top 5 rows



In [16]:
#We can also use Spark functions to create columns:
from pyspark.sql.functions import col, expr

In [17]:
col('hwy')

Column<'hwy'>

In [18]:
#we can mix-n-match our syntax to produce the same column objects made above:
avg_column = (col('hwy') + col('cty'))/2

mpg.select(
    col('hwy'). alias('highway_mileage'),
    mpg.cty.alias('city_mileage'),
    avg_column.alias('avg_mileage'),
).show(5)

+---------------+------------+-----------+
|highway_mileage|city_mileage|avg_mileage|
+---------------+------------+-----------+
|             29|          18|       23.5|
|             29|          21|       25.0|
|             31|          20|       25.5|
|             30|          21|       25.5|
|             26|          16|       21.0|
+---------------+------------+-----------+
only showing top 5 rows



In [19]:
# the expr function is similar to col but can express manipulations specific to the column within the string that defines
#the column itself:

mpg.select(
    expr("hwy"), #the same as 'col'
    expr("hwy + 1"), #aggregate
    expr("hwy AS highway_mileage"), #using alias
    expr("hwy + 1 AS highway_incremented"), #combo of above
).show(5)

+---+---------+---------------+-------------------+
|hwy|(hwy + 1)|highway_mileage|highway_incremented|
+---+---------+---------------+-------------------+
| 29|       30|             29|                 30|
| 29|       30|             29|                 30|
| 31|       32|             31|                 32|
| 30|       31|             30|                 31|
| 26|       27|             26|                 27|
+---+---------+---------------+-------------------+
only showing top 5 rows



In [20]:
#below I will write out the exact same column, just note that each is a different syntax choice:
mpg.select(
    mpg.hwy.alias("highway"),
    col("hwy").alias("highway"),
    expr("hwy").alias("highway"),
    expr("hwy AS highway"),
).show(5)

+-------+-------+-------+-------+
|highway|highway|highway|highway|
+-------+-------+-------+-------+
|     29|     29|     29|     29|
|     29|     29|     29|     29|
|     31|     31|     31|     31|
|     30|     30|     30|     30|
|     26|     26|     26|     26|
+-------+-------+-------+-------+
only showing top 5 rows

